# "ห้องปฏิบัติการเสริม - เซลล์ประสาทและชั้น
ในห้องปฏิบัติการนี้ เราจะสำรวจการทำงานภายในของเซลล์ประสาท/หน่วย และชั้น โดยเฉพาะอย่างยิ่ง ห้องปฏิบัติการจะเปรียบเทียบกับโมเดลที่คุณได้เรียนรู้ในหลักสูตร 1 ซึ่งเป็นโมเดลการถดถอย/เชิงเส้นและโมเดลโลจิสติก ห้องปฏิบัติการจะแนะนำ TensorFlow และแสดงให้เห็นว่าโมเดลเหล่านี้ถูกนำไปใช้ในกรอบงานอย่างไร"
<figure>
   <img src="./images/C2_W1_NeuronsAndLayers_v1.png"  style="width:540px;height:200px;" >
</figure>


## Packages
**Tensorflow และ Keras**  
Tensorflow เป็นแพ็คเกจการเรียนรู้ของเครื่องที่พัฒนาโดย Google ในปี 2019 Google ได้รวม Keras เข้ากับ Tensorflow และเปิดตัว Tensorflow 2.0 Keras เป็นเฟรมเวิร์กที่พัฒนาโดย François Chollet อย่างอิสระ ซึ่งสร้างอินเทอร์เฟซแบบเลเยอร์ที่เรียบง่ายสำหรับ Tensorflow หลักสูตรนี้จะใช้อินเทอร์เฟซ Keras"



In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_common.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_neurons_utils.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('lab_neurons_utils.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.activations import sigmoid
from lab_utils_common import dlc
from lab_neurons_utils import plt_prob_1d, sigmoidnp, plt_linear, plt_logistic
plt.style.use('./deeplearning.mplstyle')
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

## นิวรอนโดยไม่มีการกระตุ้น - การถดถอย/โมเดลเชิงเส้น

### ชุดข้อมูล
เราจะใช้ชุดข้อมูลตัวอย่างจากหลักสูตรแรกในการวิเคราะห์การถดถอยเชิงเส้นของราคาบ้าน

In [ ]:
X_train = np.array([[1.0], [2.0]], dtype=np.float32)           #(size in 1000 square feet)
Y_train = np.array([[300.0], [500.0]], dtype=np.float32)       #(price in 1000s of dollars)

fig, ax = plt.subplots(1,1)
ax.scatter(X_train, Y_train, marker='x', c='r', label="Data Points")
ax.legend( fontsize='xx-large')
ax.set_ylabel('Price (in 1000s of dollars)', fontsize='xx-large')
ax.set_xlabel('Size (1000 sqft)', fontsize='xx-large')
plt.show()

### การถดถอย/โมเดลเชิงเส้น
ฟังก์ชันที่ดำเนินการโดยนิวรอนที่ไม่มีการกระตุ้นนั้นเหมือนกับในหลักสูตรที่ 1 ซึ่งเป็นการถดถอยเชิงเส้น:
$$ f_{\mathbf{w},b}(x^{(i)}) = \mathbf{w}\cdot x^{(i)} + b \tag{1}$$



เราสามารถกำหนดชั้น (layer) ที่มีหนึ่งนิวรอนหรือหน่วย (unit) และเปรียบเทียบกับฟังก์ชันการถดถอยเชิงเส้นที่คุ้นเคย

In [ ]:
linear_layer = tf.keras.layers.Dense(units=1, activation = 'linear', )

Let's examine the weights.

In [ ]:
linear_layer.get_weights()

ยังไม่มีน้ำหนักเนื่องจากน้ำหนักยังไม่ได้ถูกสร้างขึ้น มาลองใช้โมเดลกับตัวอย่างหนึ่งใน  `X_train`. เพื่อกระตุ้นการสร้างน้ำหนักกัน โปรดทราบว่าอินพุตของเลเยอร์ต้องเป็น 2 มิติ ดังนั้นเราจะต้องปรับรูปร่างของมัน

In [ ]:
a1 = linear_layer(X_train[0].reshape(1,1))
print(a1)

ผลลัพธ์ที่ได้คือเทนเซอร์ (ชื่ออื่นของอาร์เรย์) ที่มีรูปร่าง (1,1) หรือหนึ่งรายการ ตอนนี้เรามาดูน้ำหนักและไบแอส น้ำหนักเหล่านี้ถูกเริ่มต้นแบบสุ่มเป็นตัวเลขขนาดเล็ก และไบแอสเริ่มต้นเป็นศูนย์

In [ ]:
w, b= linear_layer.get_weights()
print(f"w = {w}, b={b}")

โมเดลการถดถอยเชิงเส้น (1) ที่มีฟีเจอร์อินพุตเพียงตัวเดียวจะมีน้ำหนักและค่าบากเพียงตัวเดียว ซึ่งตรงกับมิติของ`linear_layer` ของเราข้างต้น  

น้ำหนักจะถูกเริ่มต้นด้วยค่าสุ่ม ดังนั้นเราจึงตั้งค่าเหล่านี้ให้เป็นค่าที่รู้จักกัน

In [ ]:
set_w = np.array([[200]])
set_b = np.array([100])

# set_weights takes a list of numpy arrays
linear_layer.set_weights([set_w, set_b])
print(linear_layer.get_weights())

มาเปรียบเทียบสมการ (1) กับผลลัพธ์ของเลเยอร์กัน

In [ ]:
a1 = linear_layer(X_train[0].reshape(1,1))
print(a1)
alin = np.dot(set_w,X_train[0].reshape(1,1)) + set_b
print(alin)

พวกมันสร้างค่าเดียวกัน!
ตอนนี้ เราสามารถใช้เลเยอร์เชิงเส้นของเราเพื่อทำการคาดการณ์เกี่ยวกับข้อมูลการฝึกของเราได้

In [ ]:
prediction_tf = linear_layer(X_train)
prediction_np = np.dot( X_train, set_w) + set_b

In [ ]:
plt_linear(X_train, Y_train, prediction_tf, prediction_np)

## Neuron with Sigmoid activation
นิวรอนที่มีการกระตุ้นซิกมอยด์ (Neuron with Sigmoid activation) เหมือนกับใน Course 1, logistic  regression:
$$ f_{\mathbf{w},b}(x^{(i)}) = g(\mathbf{w}x^{(i)} + b) \tag{2}$$
โดยที่ $$g(x) = sigmoid(x)$$ 

มาตั้งค่า $w$ และ $b$ เป็นค่าที่ทราบและตรวจสอบโมเดลกัน

### ชุดข้อมูล
เราจะใช้ ชุดข้อมูล จากหลักสูตรที่ 1 เพื่อฝึกโมเดล Logistic Regression

In [ ]:
X_train = np.array([0., 1, 2, 3, 4, 5], dtype=np.float32).reshape(-1,1)  # 2-D Matrix
Y_train = np.array([0,  0, 0, 1, 1, 1], dtype=np.float32).reshape(-1,1)  # 2-D Matrix

In [ ]:
pos = Y_train == 1
neg = Y_train == 0

fig,ax = plt.subplots(1,1,figsize=(4,3))
ax.scatter(X_train[pos], Y_train[pos], marker='x', s=80, c = 'red', label="y=1")
ax.scatter(X_train[neg], Y_train[neg], marker='o', s=100, label="y=0", facecolors='none', 
              edgecolors=dlc["dlblue"],lw=3)

ax.set_ylim(-0.08,1.1)
ax.set_ylabel('y', fontsize=12)
ax.set_xlabel('x', fontsize=12)
ax.set_title('one variable plot')
ax.legend(fontsize=12)
plt.show()

### Logistic Neuron
เราสามารถนำฟังก์ชันการกระตุ้นซิกมอยด์ (sigmoid activation) มาใช้กับ "เซลล์ประสาทโลจิสติก" (logistic neuron) ได้ ฟังก์ชันของเซลล์ประสาทนี้จะถูกอธิบายโดยสมการ (2) ด้านบน

ส่วนนี้จะสร้างโมเดล Tensorflow ที่ประกอบด้วยเลเยอร์โลจิสติกของเรา เพื่อแสดงวิธีการสร้างโมเดลแบบอื่น Tensorflow มักใช้ในการสร้างโมเดลแบบหลายชั้น และ [Sequential](https://keras.io/guides/sequential_model/) โมเดลนี้เป็นวิธีการที่สะดวกในการสร้างโมเดลเหล่านี้

In [ ]:
model = Sequential(
    [
        tf.keras.layers.Dense(1, input_dim=1,  activation = 'sigmoid', name='L1')
    ]
)

`model.summary()` แสดงเลเยอร์และจำนวนพารามิเตอร์ในโมเดล มีเลเยอร์เดียวในโมเดลนี้ และเลเยอร์นั้นมีหน่วยเดียว หน่วยนี้มีพารามิเตอร์สองตัว $w$ และ $b$.

In [ ]:
model.summary()

In [ ]:
logistic_layer = model.get_layer('L1')
w,b = logistic_layer.get_weights()
print(w,b)
print(w.shape,b.shape)

มาตั้งค่าน้ำหนักและความเอนเอียงเป็นค่าที่ทราบกันดี

In [ ]:
set_w = np.array([[2]])
set_b = np.array([-4.5])
# set_weights takes a list of numpy arrays
logistic_layer.set_weights([set_w, set_b])
print(logistic_layer.get_weights())

มาเปรียบเทียบสมการ (2) กับผลลัพธ์ของเลเยอร์กัน

In [ ]:
a1 = model.predict(X_train[0].reshape(1,1))
print(a1)
alog = sigmoidnp(np.dot(set_w,X_train[0].reshape(1,1)) + set_b)
print(alog)

พวกมันสร้างค่าเดียวกัน!ตอนนี้ เราสามารถใช้เลเยอร์โลจิสติกและโมเดล NumPy ของเราเพื่อทำการทำนายบนข้อมูลการฝึกของเราได้

In [ ]:
plt_logistic(X_train, Y_train, model, set_w, set_b, pos, neg)

การแรเงาด้านบนสะท้อนผลลัพธ์ของฟังก์ชันซิกมอยด์ (sigmoid) ซึ่งมีค่าตั้งแต่ 0 ถึง 1

# ยินดีด้วย!
คุณได้สร้างเครือข่ายประสาทเทียมที่ง่ายมากและได้สำรวจความคล้ายคลึงระหว่างเซลล์ประสาทกับการถดถอยเชิงเส้นและการถดถอยโลจิสติกจากหลักสูตร 1